In [205]:
import pandas as pd

In [206]:
data = pd.read_csv('../data/zomato.csv')

In [207]:
data.drop(columns=["dish_liked", "listed_in(type)", "listed_in(city)", 'menu_item'], inplace=True)

In [208]:
data.dropna(inplace=True)

In [209]:
mask_list = data["reviews_list"].apply(lambda x: isinstance(x, list) and len(x) == 0)
data.drop(index=data[mask_list].index, inplace=True)

In [210]:
data["rate"] = data["rate"].str.strip().str.replace("/5", "").str.replace("NEW", "0").str.replace("-", "0").astype(float)

In [211]:
data["online_order"] = data["online_order"].apply(lambda x: 1 if x == "Yes" else 0)

In [212]:
data["book_table"] = data["book_table"].apply(lambda x: 1 if x == "Yes" else 0)

In [213]:
data["approx_cost(for two people)"] = data["approx_cost(for two people)"].str.replace(",", "").astype(float)

In [225]:
def clean_phone_numbers(phone_str):
  phones = phone_str.replace("\n", ", ").replace("\r", "").strip()
  phone_numbers = [num.strip() for num in phones.split(",") if num.strip()]
  phone_numbers = list(set(phone_numbers))
  return phone_numbers[0] if len(phone_numbers) == 1 else phone_numbers

new_data = data["phone"].apply(clean_phone_numbers)

data["phone"] = new_data

In [226]:
data.reset_index(drop=True, inplace=True)

In [ ]:
def normalize_reviews(review_row):
  splittled_data = review_row.split("), (")
  splittled_data = list(set(splittled_data))
  
  result = []
  for i in range(len(splittled_data)):
    try:
      splittled_data[i] = splittled_data[i].replace("[(", "").replace(")]", "").replace("RATED\\n  ", "")
      review_parts = splittled_data[i].split("', '")
      rating_str = review_parts[0].replace("'", "").replace("Rated ", "").strip()
      review_parts[0] = float(rating_str)
      
      result.append(review_parts)
    except (ValueError, IndexError):
      continue
  
  return result
  
new_data = data["reviews_list"].apply(normalize_reviews)
data["reviews_list"] = new_data

In [243]:
data.to_csv("../data/zomato_cleaned.csv", index=False)